In [1]:

!pip install numba

In [2]:
import pandas as pd
import numpy as np

In [38]:
from numba import cuda
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc

from torch.optim import AdamW
import torch.cuda
from google.colab import runtime
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Preprocessing

In [95]:
dc = pd.read_csv('/content/DecOp_data_EN_500.csv', sep=';', encoding = "ISO-8859-1")
dc.head()

d1 = []
for i in range(dc.shape[0]):
  row = dc.iloc[i]
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['A'].replace('\n', " ") ,
  'labels'  : row['GT.A']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['E'].replace('\n', " ")  ,
  'labels'  : row['GT.E']})
  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['GM'].replace('\n', " ")  ,
  'labels'  : row['GT.GM']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['Pom'].replace('\n', " ")  ,
  'labels'  : row['GT.Pom']})

  d1.append({'ID': row['ID'],
      'age' :row['age'], 'gender': row['gender'],
  'sent' : row['CL'].replace('\n', " ")  ,
  'labels'  : row['GT.CL']})

decop = pd.DataFrame.from_records(d1)

decop=decop[['ID', 'sent', 'labels']]
decop['type']='A'

In [96]:
hc = pd.read_csv('/content/preprocessed_hcV3.csv', sep=';', encoding = "ISO-8859-1")

mem = hc[hc['memType']!='retold']
mem = mem.dropna(subset=['story', 'memType'])
mem = mem[['story', 'memType']]
mem['memType'][mem['memType']=='recalled']='T'
mem['memType'][mem['memType']=='imagined']='F'
mem = mem.rename(columns={'story': 'sent', 'memType': 'labels'})
mem['type']='B'

In [97]:
intent = pd.read_csv('/content/sign_events_data_statements.csv', encoding = "ISO-8859-1")


intent['outcome_class'][intent['outcome_class']=='t']='T'
intent['outcome_class'][intent['outcome_class']=='d']='F'
intent['q1']=intent['q1'].apply(lambda x: x.replace('\n', ''))
intent = intent.rename(columns={'q1': 'sent', 'outcome_class': 'labels'})
intent = intent[['sent', 'labels']]
intent['type']='C'

<ipython-input-97-ba4812cfd6fa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intent['outcome_class'][intent['outcome_class']=='t']='T'
<ipython-input-97-ba4812cfd6fa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  intent['outcome_class'][intent['outcome_class']=='d']='F'


In [98]:
def create_dataset(Scenario = {'train':[], 'test':[], 'scenario':1}, decop=None, mem = None, intent = None, seed=42, cv=10):

  train={f'split_{i+1}':[] for i in range(cv)}
  test={f'split_{i+1}':[] for i in range(cv)}

  np.random.seed(seed)

  de_arr = np.arange(1, 501)
  np.random.shuffle(de_arr)
  de_arr_r = de_arr.reshape((cv, int(500/cv)))

  num_row = (mem.index.shape[0] - (mem.shape[0]%cv))
  me_arr = np.random.choice(mem.index.shape[0], int(num_row), replace=False)
  me_arr_r = me_arr.reshape((cv, int(num_row/cv)))

  in_arr = np.arange(0, intent.index.shape[0])
  np.random.shuffle(in_arr)
  in_arr_r = in_arr.reshape((cv, int(intent.shape[0]/cv))) ### fin qui testata

  cv_ind = 0
  for k in train.keys():



    if Scenario['scenario'] == 1 or Scenario['scenario'] == 3:

      if 'decop' in Scenario['train']:
        ind = de_arr_r[cv_ind,:]
        antind = [i for i in de_arr if i not in ind]

        decop_train = decop[decop['ID'].isin(antind)]
        decop_test = decop[decop['ID'].isin(ind)]

        train[k].append(decop_train)
        test[k].append(decop_test)

      if 'hyppocorpus' in Scenario['train']:
        ind = me_arr_r[cv_ind,:]
        antind = [i for i in me_arr if i not in ind]

        mem_train = mem[mem.index.isin(antind)]
        mem_test = mem[mem.index.isin(ind)]

        train[k].append(mem_train)
        test[k].append(mem_test)

      if 'intention' in Scenario['train']:

        ind = in_arr_r[cv_ind,:]
        antind = [i for i in in_arr if i not in ind]

        intent_train = intent[intent.index.isin(antind)]
        intent_test = intent[intent.index.isin(ind)]

        train[k].append(intent_train)
        test[k].append(intent_test)

      train[k] = pd.concat(train[k])
      test[k] = pd.concat(test[k])

    else:

      if 'decop' in Scenario['train']:
        train[k].append(decop.copy())
      elif 'decop' in Scenario['test']:
        test[k].append(decop.copy())

      if 'hyppocorpus' in Scenario['train']:
        train[k].append(mem.copy())
      elif 'hyppocorpus' in Scenario['test']:
        test[k].append(mem.copy())

      if 'intention' in Scenario['train']:
        train[k].append(intent.copy())
      elif 'intention' in Scenario['test']:
        test[k].append(intent.copy())

      train[k] = pd.concat(train[k])
      test[k] = test[k][0]

    cv_ind+=1



  return train, test



In [100]:
seed=42
np.random.seed(seed)
collect_result={}

results={}
types={}
preds={}
trus={}
scenarios = [{'train': ['decop'], 'test': ['decop'],  'scenario': 1},
             {'train': ['hyppocorpus'], 'test': ['hyppocorpus'],  'scenario': 1},
             {'train': ['intention'], 'test': ['intention'],  'scenario': 1},
             {'train': ['hyppocorpus', 'intention'], 'test': ['decop'],  'scenario': 2},
             {'train': ['decop', 'intention'], 'test': ['hyppocorpus'],  'scenario': 2},
             {'train': ['decop', 'hyppocorpus'], 'test': ['intention'],  'scenario': 2},
             {'train': ['decop', 'hyppocorpus', 'intention'], 'test': ['decop', 'hyppocorpus', 'intention'],  'scenario': 3}
             ]
for scen in scenarios:
  if scen['scenario']==2:
    numcv=1
  else:
    numcv=10
  training, test = create_dataset(scen, decop=decop, mem=mem, intent=intent, seed=seed, cv=numcv)
  i =0
  accuracy = []
  for sp in tqdm(training.keys()):
    with warnings.catch_warnings():
      warnings.simplefilter("ignore")
      training[sp]['labels'] = training[sp]['labels'].map({'F':1, 'T':0})
      test[sp]['labels'] = test[sp]['labels'].map({'F':1, 'T':0})

      training[sp] = training[sp].drop('type', axis=1)
      test[sp] = test[sp].drop('type', axis=1)

      if i==0:
        full_data = pd.concat([training[sp], test[sp]])
        i=1
      vectorizer = CountVectorizer()
      bag = vectorizer.fit(full_data['sent'].to_list())

      x_train = bag.transform(training[sp]['sent'].to_list())
      x_test = bag.transform(test[sp]['sent'].to_list())

      y_train = training[sp]['labels']
      y_test = test[sp]['labels']

      lr = LogisticRegression()

      lr.fit(x_train, y_train)

      pred = lr.predict(x_test)

      accuracy.append((pred==y_test).mean())
  print('########################')
  print(scen)
  print(np.array(accuracy).mean())
  print(np.array(accuracy).std())
  print('########################')


100%|██████████| 10/10 [00:04<00:00,  2.35it/s]


########################
{'train': ['decop'], 'test': ['decop'], 'scenario': 1}
0.7615999999999999
0.02902137143554729
########################


100%|██████████| 10/10 [00:03<00:00,  3.10it/s]


########################
{'train': ['hyppocorpus'], 'test': ['hyppocorpus'], 'scenario': 1}
0.5757575757575759
0.07666127661014253
########################


100%|██████████| 10/10 [00:03<00:00,  2.53it/s]


...

########################
{'train': ['decop', 'intention'], 'test': ['hyppocorpus'], 'scenario': 2}
0.5654761904761905
0.0
########################


100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


########################
{'train': ['decop', 'hyppocorpus'], 'test': ['intention'], 'scenario': 2}
0.49573170731707317
0.0
########################


100%|██████████| 10/10 [00:13<00:00,  1.37s/it]

########################
{'train': ['decop', 'hyppocorpus', 'intention'], 'test': ['decop', 'hyppocorpus', 'intention'], 'scenario': 3}
0.6885906040268457
0.02254518601761813
########################
